# 1.05 Mixed Layer Decorrelation and Residence Time

---

Author: Riley X. Brady

Date: 11/19/2020

---

_Note_: This code could be a lot cleaner, but it gets the job done.

This calculates the decorrelation timescale of DIC once it enters the mixed layer (200 m). We first select the ensemble of particles based on where they first cross 200 m after their last 1000 m crossing. Then for each particle, we evaluate the remainder of the time series following that mixed layer crossing into the given topographic (or non-topographic) region. We assess every single 200 m crossing by the particle, including the first crossing into 200 m, and calculate the decorrelation time scale of DIC during that time.

We account for that decorrelation time scale if:

* The 200 m crossing happens outside of the annual sea ice edge
* The 200 m crossing happens south of 45S
* The 200 m crossing happens in waters deeper than 500 m (to avoid shelf-trapped particles)
* The autocorrelation is significant with p < 0.05

We also discard the given decorrelation time computed if the particle is still above 200 m when the simulation ends.

For residence time, we use the same principles, but still clock the residence time even if the decorrelation is not statistically significant.

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2
import figutils
import numpy as np
import xarray as xr
from scipy.stats import pearsonr

from dask.distributed import Client

In [2]:
import scipy

print(f"numpy: {np.__version__}")
print(f"xarray: {xr.__version__}")
print(f"scipy: {scipy.__version__}")

numpy: 1.19.4
xarray: 0.16.1
scipy: 1.5.3


In [3]:
# This is my TCP client from the `launch_cluster` notebook. I use it
# for distributed computing with `dask` on NCAR's machine, Casper.
client = Client("tcp://...")

Load in information from the Eulerian mesh that will be used when calculating decorrelation and residence time.

In [4]:
ice = (
    xr.open_dataset("../data/eulerian_sea_ice_climatology.nc").mean("month").icePresent
)
mesh = xr.open_dataset("../data/mesh.nc")
depth = xr.open_dataset("../data/bottomDepth.nc")

mpas_lat = np.rad2deg(mesh.latCell)
mpas_lon = np.rad2deg(mesh.lonCell)
bottomDepth = depth.bottomDepth

# To save some cost, only look S of 45S
# These will be global variables used as a reference in the
# decorrelation and residence time calculation.
SEA_ICE = ice.where(mpas_lat < -45, drop=True)
BOTTOM_DEPTH = bottomDepth.where(mpas_lat < -45, drop=True)
MPAS_LON = mpas_lon.where(mpas_lat < -45, drop=True)
MPAS_LAT = mpas_lat.where(mpas_lat < -45, drop=True)

We'll also load in all of the deep upwelled particles as a base from which we will subset our ensembles.

**Note**: I loaded in the netCDF file, and chunked it, and then saved it back out as a `zarr` file. This makes `dask` run a lot more efficiently. E.g.,

```python
ds = xr.open_dataset('../data/southern_ocean_deep_upwelling_particles.nc')
ds = ds.chunk({'time': -1, 'nParticles': 'auto'})
ds.to_zarr('../data/southern_ocean_deep_upwelling_particles.zarr', consolidated=True)
```

You could probably chunk the particles into slightly smaller chunks for even faster performance.

In [5]:
# Load in the `zarr` file, which is pre-chunked and already has been
# filtered from the original 1,000,000 particles to the 19,002 that
# upwell last across 1000 m S of 45S and outside of the annual sea ice
# edge.
filepath = "../data/southern_ocean_deep_upwelling_particles.zarr/"
ds = xr.open_zarr(filepath, consolidated=True)

In [6]:
ds = ds.chunk({"time": -1, "nParticles": 5000})
ds = ds.persist()

Now we define all of the functions that will be used to calculate the decorrelation and residence time. These could definitely be cleaned up, but I kind of just want to get this paper submitted! I have found that `apply_ufunc` is sluggish when calling functions that exist in an external `.py` script, so I just define them all here.

In [7]:
def _pearsonr_by_hand(x, lag):
    """Calculate the pearson r correlation for autocorrelation.

    x : Time series to calculate autocorrelation for (particleDIC).
    lag : int of the lag for which to compute the autocorrelation.
    """
    y1 = x[: (len(x) - lag)]
    y2 = x[lag:]
    if len(y1) >= 2:
        r, p = pearsonr(y1, y2)
        return r, p
    else:
        # can't compute autocorrelation for 2 points or less.
        return np.nan, np.nan


def _decorrelation_time(tracer):
    """Computes decorrelation time (in days) based on the e-folding time.

    If p > 0.05, don't return.

    tracer : particleDIC or any other tracer for which to compute decorrelation.
    """
    # Find decorrelation time.
    auto = np.array([_pearsonr_by_hand(tracer, lag) for lag in np.arange(len(tracer))])
    # extract corrcoef and p value.
    r = auto[:, 0]
    p = auto[:, 1]
    e_folding = (r <= 1 / np.e).argmax()
    if p[e_folding] < 0.05:
        decorr_time = int(e_folding * 2)
        return decorr_time
    else:
        # don't return if non-significant correlation.
        return np.nan


def _find_mpas_cell(xParticle, yParticle):
    """Returns the idx to plug into the MPAS mask array
    from above to check whether to keep or not.

    We use global MPAS_LON and MPAS_LAT variables here to avoid
    having too long of a function signature and passing a bunch of
    stuff in and out of apply_ufunc.

    xParticle: lonParticle (degrees)
    yParticle: latParticle (degrees)
    """
    dx = MPAS_LON - xParticle
    dy = MPAS_LAT - yParticle
    diff = abs(dx) + abs(dy)
    idx = np.nanargmin(diff)
    return idx


def _compute_idx_of_first_200m_crossing(z):
    """Find first time particle upwells across 200 m.

    z : zLevelParticle
    """
    currentDepth = z
    previousDepth = np.roll(z, 1)
    previousDepth[0] = 999
    cond = (currentDepth >= -200) & (previousDepth < -200)
    idx = cond.argmax()
    return idx


def _compute_idx_of_last_1000m_crossing(z):
    """Find index of final time particle upwells across 1000 m.

    z : zLevelParticle
    """
    currentDepth = z
    previousDepth = np.roll(z, 1)
    previousDepth[0] = 999  # So we're not dealing with a nan here.
    cond = (currentDepth >= -1000) & (previousDepth < -1000)
    idx = (
        len(cond) - np.flip(cond).argmax() - 1
    )  # Finds last location that condition is true.
    return idx


def mixed_layer_decorrelation_time(x, y, z, DIC):
    """Computes the decorrelation time of DIC during the given mixed layer stay.

    * Makes sure that the 200 m crossing happens S of 45S
    * Makes sure that e-folding autocorrelation coefficient has p < 0.05.
    * Makes sure that the given 200 m crossing happens outside of the annual sea ice zone.
    * Makes sure that the bottom depth at the given crossing is > 500 m, to avoid coastally
      trapped particles that are just oscillating around the mixed layer.
    * Throws away the decorrelation if the simulation ends and it's still above 200 m.

    x : lonParticle (degrees)
    y : latParticle (degrees)
    z : zLevelParticle * -1 (m)
    DIC : particleDIC
    """
    # Conservative estimate on the max number of 200 m crossings a given particle
    # could have. Will fill in one value for each crossing, if applicable.
    MIXED_LAYER_DECORRELATION_TIME = np.zeros(200)
    MIXED_LAYER_DECORRELATION_TIME[:] = np.nan

    # Subset from final 1000 m crossing and beyond.
    idx_a = _compute_idx_of_last_1000m_crossing(z * -1)
    x = x[idx_a - 1 : :]
    y = y[idx_a - 1 : :]
    z = z[idx_a - 1 : :]
    DIC = DIC[idx_a - 1 : :]

    # Find first 200 m crossing after that and subset to
    # remainder of trajectory after this.
    idx_b = _compute_idx_of_first_200m_crossing(z * -1)
    x = x[idx_b - 1 : :]
    y = y[idx_b - 1 : :]
    z = z[idx_b - 1 : :]
    DIC = DIC[idx_b - 1 : :]

    # Now we analyze all 200m crossings from this point and beyond.
    previous_depth = np.roll(z, 1)

    # Don't include first time step since we don't know
    # where it was before.
    previous_depth = previous_depth[1::]
    current_depth = z[1::]

    # Find indices where particle upwells into 200m. Looking for all occurrences.
    (mixed_layer_idxs,) = np.where((previous_depth > 200) & (current_depth < 200))
    # Account for `np.roll(...)`
    mixed_layer_idxs += 1

    # Only maintain those that upwell S of 45S
    mixed_layer_idxs = mixed_layer_idxs[y[mixed_layer_idxs] < -45]

    for filler, idx in enumerate(mixed_layer_idxs):
        cellidx = _find_mpas_cell(x[idx], y[idx])

        # Check that particle crosses into 200 m outside of sea ice zone
        # and in waters deeper than 500 m.
        if (SEA_ICE[cellidx] < 0.75) and (BOTTOM_DEPTH[cellidx] > 500):
            zsubset = z[idx::]
            dicsubset = DIC[idx::]
            time_steps_below_mixed_layer = np.argwhere(zsubset > 200)

            # If this isn't True, it stays above 200m for remainder of trajectory
            # and we toss it away, leaving it as a NaN.
            if time_steps_below_mixed_layer.any():
                idx_of_next_subduction = time_steps_below_mixed_layer.min()

                mixed_layer_dic_subset = dicsubset[0:idx_of_next_subduction]
                # returns integer number of days for decorr time
                # (e-folding decorrelation time * 2 days per time step on average)
                decorr = _decorrelation_time(mixed_layer_dic_subset)

                # Not possible since at time step 0 it's exactly 1. This just
                # means it didn't decorr over how long it was up here.
                if decorr != 0:
                    MIXED_LAYER_DECORRELATION_TIME[filler] = decorr
    return MIXED_LAYER_DECORRELATION_TIME


def mixed_layer_residence_time(x, y, z):
    """Computes the residence time of a particle during the given mixed layer stay.

    * Makes sure that the 200 m crossing happens S of 45S
    * Makes sure that e-folding autocorrelation coefficient has p < 0.05.
    * Makes sure that the given 200 m crossing happens outside of the annual sea ice zone.
    * Makes sure that the bottom depth at the given crossing is > 500 m, to avoid coastally
      trapped particles that are just oscillating around the mixed layer.
    * Throws away the decorrelation if the simulation ends and it's still above 200 m.

    x : lonParticle (degrees)
    y : latParticle (degrees)
    z : zLevelParticle * -1 (m)
    """
    # Conservative estimate on the max number of 200 m crossings a given particle
    # could have. Will fill in one value for each crossing, if applicable.
    MIXED_LAYER_RESIDENCE_TIME = np.zeros(200)
    MIXED_LAYER_RESIDENCE_TIME[:] = np.nan

    # Subset from final 1000 m crossing and beyond.
    idx_a = _compute_idx_of_last_1000m_crossing(z * -1)
    x = x[idx_a - 1 : :]
    y = y[idx_a - 1 : :]
    z = z[idx_a - 1 : :]

    # Find first 200 m crossing after that and subset to
    # remainder of trajectory after this.
    idx_b = _compute_idx_of_first_200m_crossing(z * -1)
    x = x[idx_b - 1 : :]
    y = y[idx_b - 1 : :]
    z = z[idx_b - 1 : :]

    # Now we analyze all 200m crossings from this point and beyond.
    previous_depth = np.roll(z, 1)

    # Don't include first time step since we don't know
    # where it was before.
    previous_depth = previous_depth[1::]
    current_depth = z[1::]

    # Find indices where particle upwells into 200m. Looking for all occurrences.
    (mixed_layer_idxs,) = np.where((previous_depth > 200) & (current_depth < 200))
    # Account for `np.roll(...)`
    mixed_layer_idxs += 1

    # Only maintain those that upwell S of 45S
    mixed_layer_idxs = mixed_layer_idxs[y[mixed_layer_idxs] < -45]

    for filler, idx in enumerate(mixed_layer_idxs):
        cellidx = _find_mpas_cell(x[idx], y[idx])

        # Check that particle crosses into 200 m outside of sea ice zone
        # and in waters deeper than 500 m.
        if (SEA_ICE[cellidx] < 0.75) and (BOTTOM_DEPTH[cellidx] > 500):
            zsubset = z[idx::]
            time_steps_below_mixed_layer = np.argwhere(zsubset > 200)

            # If this isn't True, it stays above 200m for remainder of trajectory
            # and we toss it away, leaving it as a NaN.
            if time_steps_below_mixed_layer.any():
                idx_of_next_subduction = time_steps_below_mixed_layer.min()
                mixed_layer_z_subset = zsubset[0:idx_of_next_subduction]
                MIXED_LAYER_RESIDENCE_TIME[filler] = int(len(mixed_layer_z_subset) * 2)
    return MIXED_LAYER_RESIDENCE_TIME

## Decorrelation Time Calculations

---

Topographic Regions

In [8]:
crossings = xr.open_dataset("../data/postproc/200m.crossing.locations.nc")
xc, yc = crossings["lon_crossing"], crossings["lat_crossing"]

for region in ["drake", "crozet", "kerguelan", "campbell"]:
    print(f"{region}...")
    x0, x1, y0, y1 = figutils.BOUNDS[region]
    if region == "drake":
        x0 += 360
        x1 += 360

    conditions = (xc > x0) & (xc < x1) & (yc > y0) & (yc < y1)
    particle_ids = conditions.where(conditions, drop=True).nParticles.astype(int)

    # Select ensemble based on 200m crossing location.
    ensemble = ds.sel(nParticles=particle_ids)
    ensemble = ensemble.chunk({"time": -1, "nParticles": 250}).persist()

    # Add some helpful variables
    ensemble["depth"] = ensemble.zLevelParticle * -1
    ensemble["latDegrees"] = np.rad2deg(ensemble.latParticle)
    ensemble["lonDegrees"] = np.rad2deg(ensemble.lonParticle)

    # Calculate decorrelation time for every mixed layer instance
    # following that first 200 m crossing.
    decorr_result = xr.apply_ufunc(
        mixed_layer_decorrelation_time,
        ensemble.lonDegrees,
        ensemble.latDegrees,
        ensemble.depth,
        ensemble.particleDIC,
        input_core_dims=[["time"], ["time"], ["time"], ["time"]],
        output_core_dims=[["crossings"]],
        vectorize=True,
        dask="parallelized",
        dask_gufunc_kwargs={"output_sizes": {"crossings": 200}},
        output_dtypes=[float],
    )

    %time decorr_result = decorr_result.compute()

    # Create single dimension of all crossings for the given ensemble.
    decorr_result = decorr_result.stack(
        all_crossings=["nParticles", "crossings"]
    ).dropna("all_crossings")
    decorr_result = decorr_result.rename("decorr").to_dataset()
    decorr_result.attrs[
        "description"
    ] = "surface DIC decorrelation time for every 200 m crossing after the first mixed layer crossing for the given particle ensemble."
    decorr_result.attrs[
        "dropped_cases"
    ] = "inside sea ice zone; N of 45S; p > 0.05 autocorrelation; in waters shallower than 500m; simulation ends with particle above 200m"
    decorr_result.reset_index("all_crossings").to_netcdf(
        f"../data/postproc/{region}.DIC.decorr.nc"
    )

drake...
CPU times: user 238 ms, sys: 50.9 ms, total: 289 ms
Wall time: 27.2 s
crozet...
CPU times: user 104 ms, sys: 41 ms, total: 145 ms
Wall time: 26.3 s
kerguelan...
CPU times: user 138 ms, sys: 31.3 ms, total: 169 ms
Wall time: 24.9 s
campbell...
CPU times: user 83.5 ms, sys: 29 ms, total: 112 ms
Wall time: 19.4 s


Non-Topographic Regions

In [9]:
base_conditions = crossings.nParticles < 0  # just creates an all False bool.
for region in ["drake", "crozet", "kerguelan", "campbell"]:
    x0, x1, y0, y1 = figutils.BOUNDS[region]
    if region == "drake":
        x0 += 360
        x1 += 360

    conditions = (xc > x0) & (xc < x1) & (yc > y0) & (yc < y1)
    base_conditions = base_conditions + conditions

# Used the above as a quick way to get at the particle IDs for the non-topographic
# particles.
particle_ids = crossings.where(~base_conditions, drop=True).nParticles.astype(int)
ensemble = ds.sel(nParticles=particle_ids)
ensemble = ensemble.chunk({"time": -1, "nParticles": 250}).persist()

# Add some helpful variables
ensemble["depth"] = ensemble.zLevelParticle * -1
ensemble["latDegrees"] = np.rad2deg(ensemble.latParticle)
ensemble["lonDegrees"] = np.rad2deg(ensemble.lonParticle)

# Calculate decorrelation time for every mixed layer instance
# following that first 200 m crossing.
decorr_result = xr.apply_ufunc(
    mixed_layer_decorrelation_time,
    ensemble.lonDegrees,
    ensemble.latDegrees,
    ensemble.depth,
    ensemble.particleDIC,
    input_core_dims=[["time"], ["time"], ["time"], ["time"]],
    output_core_dims=[["crossings"]],
    vectorize=True,
    dask="parallelized",
    dask_gufunc_kwargs={"output_sizes": {"crossings": 200}},
    output_dtypes=[float],
)

%time decorr_result = decorr_result.compute()

# Create single dimension of all crossings for the given ensemble.
decorr_result = decorr_result.stack(all_crossings=["nParticles", "crossings"]).dropna(
    "all_crossings"
)
decorr_result = decorr_result.rename("decorr").to_dataset()
decorr_result.attrs[
    "description"
] = "surface DIC decorrelation time for every 200 m crossing after the first mixed layer crossing for the given particle ensemble."
decorr_result.attrs[
    "dropped_cases"
] = "inside sea ice zone; N of 45S; p > 0.05 autocorrelation; in waters shallower than 500m; simulation ends with particle above 200m"
decorr_result.reset_index("all_crossings").to_netcdf(
    "../data/postproc/non_topographic.DIC.decorr.nc"
)

CPU times: user 243 ms, sys: 82.1 ms, total: 326 ms
Wall time: 34.9 s


## Residence Time Calculations 

---

Topographic regions

In [10]:
crossings = xr.open_dataset("../data/postproc/200m.crossing.locations.nc")
xc, yc = crossings["lon_crossing"], crossings["lat_crossing"]

for region in ["drake", "crozet", "kerguelan", "campbell"]:
    print(f"{region}...")
    x0, x1, y0, y1 = figutils.BOUNDS[region]
    if region == "drake":
        x0 += 360
        x1 += 360

    conditions = (xc > x0) & (xc < x1) & (yc > y0) & (yc < y1)
    particle_ids = conditions.where(conditions, drop=True).nParticles.astype(int)

    # Select ensemble based on 200m crossing location.
    ensemble = ds.sel(nParticles=particle_ids)
    ensemble = ensemble.chunk({"time": -1, "nParticles": 250}).persist()

    # Add some helpful variables
    ensemble["depth"] = ensemble.zLevelParticle * -1
    ensemble["latDegrees"] = np.rad2deg(ensemble.latParticle)
    ensemble["lonDegrees"] = np.rad2deg(ensemble.lonParticle)

    # Calculate residence time for every mixed layer instance
    # following that first 200 m crossing.
    tau_result = xr.apply_ufunc(
        mixed_layer_residence_time,
        ensemble.lonDegrees,
        ensemble.latDegrees,
        ensemble.depth,
        input_core_dims=[
            ["time"],
            ["time"],
            ["time"],
        ],
        output_core_dims=[["crossings"]],
        vectorize=True,
        dask="parallelized",
        dask_gufunc_kwargs={"output_sizes": {"crossings": 200}},
        output_dtypes=[float],
    )

    %time tau_result = tau_result.compute()

    # Create single dimension of all crossings for the given ensemble.
    tau_result = tau_result.stack(all_crossings=["nParticles", "crossings"]).dropna(
        "all_crossings"
    )
    tau_result = tau_result.rename("tau").to_dataset()
    tau_result.attrs[
        "description"
    ] = "mixed layer residence time for every 200 m crossing after the first mixed layer crossing for the given particle ensemble."
    tau_result.attrs[
        "dropped_cases"
    ] = "inside sea ice zone; N of 45S; in waters shallower than 500m; simulation ends with particle above 200m"
    tau_result.reset_index("all_crossings").to_netcdf(
        f"../data/postproc/{region}.tau.surface.nc"
    )

drake...
CPU times: user 183 ms, sys: 57.5 ms, total: 241 ms
Wall time: 19.4 s
crozet...
CPU times: user 82.9 ms, sys: 26.1 ms, total: 109 ms
Wall time: 13.6 s
kerguelan...
CPU times: user 119 ms, sys: 30.8 ms, total: 150 ms
Wall time: 14.9 s
campbell...
CPU times: user 71.6 ms, sys: 23.9 ms, total: 95.5 ms
Wall time: 12.1 s


Non-topographic regions

In [11]:
base_conditions = crossings.nParticles < 0  # just creates an all False bool.
for region in ["drake", "crozet", "kerguelan", "campbell"]:
    x0, x1, y0, y1 = figutils.BOUNDS[region]
    if region == "drake":
        x0 += 360
        x1 += 360

    conditions = (xc > x0) & (xc < x1) & (yc > y0) & (yc < y1)
    base_conditions = base_conditions + conditions

# Used the above as a quick way to get at the particle IDs for the non-topographic
# particles.
particle_ids = crossings.where(~base_conditions, drop=True).nParticles.astype(int)
# Select ensemble based on 200m crossing location.
ensemble = ds.sel(nParticles=particle_ids)
ensemble = ensemble.chunk({"time": -1, "nParticles": 250}).persist()

# Add some helpful variables
ensemble["depth"] = ensemble.zLevelParticle * -1
ensemble["latDegrees"] = np.rad2deg(ensemble.latParticle)
ensemble["lonDegrees"] = np.rad2deg(ensemble.lonParticle)

# Calculate residence time for every mixed layer instance
# following that first 200 m crossing.
tau_result = xr.apply_ufunc(
    mixed_layer_residence_time,
    ensemble.lonDegrees,
    ensemble.latDegrees,
    ensemble.depth,
    input_core_dims=[
        ["time"],
        ["time"],
        ["time"],
    ],
    output_core_dims=[["crossings"]],
    vectorize=True,
    dask="parallelized",
    dask_gufunc_kwargs={"output_sizes": {"crossings": 200}},
    output_dtypes=[float],
)

%time tau_result = tau_result.compute()

# Create single dimension of all crossings for the given ensemble.
tau_result = tau_result.stack(all_crossings=["nParticles", "crossings"]).dropna(
    "all_crossings"
)
tau_result = tau_result.rename("tau").to_dataset()
tau_result.attrs[
    "description"
] = "mixed layer residence time for every 200 m crossing after the first mixed layer crossing for the given particle ensemble."
tau_result.attrs[
    "dropped_cases"
] = "inside sea ice zone; N of 45S; in waters shallower than 500m; simulation ends with particle above 200m"
tau_result.reset_index("all_crossings").to_netcdf(
    f"../data/postproc/non_topographic.tau.surface.nc"
)

CPU times: user 216 ms, sys: 68.1 ms, total: 284 ms
Wall time: 22.7 s
